In [2]:
print("hello world")

hello world


In [21]:
import numpy as np
import matplotlib as plt

import pycuda.driver as cuda_driver
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray

import IPythonMagic
from Timer import Timer

In [9]:
%setup_logging

Global logger already initialized!


In [10]:
%cuda_context_handler context

Registering context in user workspace
Context already registered! Ignoring


In [37]:
kernel_src = """
__global__ void shmemReduction(float* output,float* input,int size) {
    // First we stride through global memory and compute 
    // the maximum for every thread
    int gid = blockIdx.x*blockDim.x + threadIdx.x; // blockIdx.x is always zero because we use just one block
    
    float max_value = -99999999.99;
    for(int i = threadIdx.x; i < size; i = i + blockDim.x) {
        max_value = fmaxf(max_value, input[i]);
    }
    // Store the per-thread maximum in temporary
    output[threadIdx.x] = max_value;
    // Store the per-thread maximum in shared memory
    __shared__ float max_shared[64];
    max_shared[threadIdx.x] = max_value;
    // Synchronize so that all thread see the same shared memory
    __syncthreads();
    // Find the maximum in shared memory
    if(threadIdx.x < 64) {
        // Reduce from 128 to 64
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+64]);
    }
    __syncthreads();
    if(threadIdx.x < 32) {
        // Reduce from 64 to 32
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+32]);
    }
    //since we have more than one active warp(threadIdx.x>32) we need to ensure that first they have finished
    __syncthreads();
    if(threadIdx.x < 16) {
        // Reduce from 32 to 16
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+16]);
    }
    if(threadIdx.x < 8) {
        // Reduce from 16 to 8
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+8]);
    }
    if(threadIdx.x < 4) {
        // Reduce from 8 to 4
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+4]);
    }
    if(threadIdx.x < 2) {
        // Reduce from 4 to 2
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+2]);
    }
    if(threadIdx.x < 1) {
        // Reduce from 2 to 1
        max_shared[threadIdx.x] = fmaxf(max_shared[threadIdx.x],max_shared[threadIdx.x+1]);
    }
    
    if(threadIdx.x == 0) {
        output[0] = max_shared[0];
    }
    
}
"""

kernel_module = cuda_compiler.SourceModule(kernel_src)
kernel_function = kernel_module.get_function("shmemReduction")

/home/ubuntu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(6): warning: variable "gid" was declared but never referenced




In [42]:
n = 128;
a = np.random.rand(n).astype(np.float32)
print(a)

a_g = GPUArray(a.shape,dtype=np.float32)
a_g.set(a)

num_thread = 128 #Each kernel will compute the maximum between two numbers
b = np.empty((1,num_thread)).astype(np.float32)
b_g = GPUArray(b.shape,dtype=np.float32)

[0.0415197  0.27639794 0.6346874  0.79028094 0.530032   0.4325671
 0.7446556  0.9761816  0.3398487  0.34358472 0.57067627 0.83579314
 0.7921029  0.09663699 0.8544314  0.96140355 0.96576345 0.30608365
 0.34680593 0.342454   0.4197055  0.8163652  0.76615626 0.9654034
 0.6327166  0.9994929  0.07451115 0.78208333 0.17395048 0.87565994
 0.604195   0.9833372  0.8590598  0.7846729  0.8799296  0.03161197
 0.39415574 0.2683006  0.57116646 0.17993945 0.6320698  0.00694536
 0.831777   0.6938064  0.12514924 0.15839885 0.23094083 0.76098853
 0.5474812  0.31646284 0.987259   0.77799374 0.682354   0.7813384
 0.30107498 0.15996346 0.87682545 0.24000433 0.02706377 0.72064304
 0.84669864 0.2500792  0.8921426  0.22878674 0.07943839 0.4254837
 0.11947915 0.8905313  0.76027364 0.65806836 0.561684   0.66613823
 0.9329324  0.8083595  0.6425296  0.18666573 0.24283965 0.6791122
 0.02231729 0.0996299  0.97998476 0.26803094 0.8298856  0.74105275
 0.41594905 0.4402502  0.7306058  0.50981456 0.06163519 0.94104934


LogicError: cuMemAlloc failed: an illegal memory access was encountered

In [41]:
block_size = (num_thread,1,1)
grid_size = (1,1,1)

kernel_function(b_g,a_g,np.int32(n),grid=grid_size,block=block_size)

b_g.get(b)

print(a)
print(b)
print(np.max(a))

LogicError: cuMemcpyDtoH failed: an illegal memory access was encountered